In [1]:
import pickle
import matplotlib.pyplot as plt
from hts import HTSRegressor
from hts.hierarchy import HierarchyTree
from m5 import *
import time
import json
from sklearn.model_selection import train_test_split
from statistics import mean

import warnings
warnings.filterwarnings("ignore")

## Auxiliar Function

In [2]:
# Function to reduce memory to increase performance to decrease run time.
def downcasting(df, verbose=False):
    '''
    reduce memory usage by downcasting data types
    from https://www.kaggle.com/harupy/m5-baseline
    '''
    
    start_mem = df.memory_usage().sum() / 1024 ** 2
    int_columns = df.select_dtypes(include=["int"]).columns
    float_columns = df.select_dtypes(include=["float"]).columns

    for col in int_columns:
        df[col] = pd.to_numeric(df[col], downcast="integer")

    for col in float_columns:
        df[col] = pd.to_numeric(df[col], downcast="float")

    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [3]:
def runModel(modelName, hierarchy, dataFrame, revisionMethod=None):
    
    clf = HTSRegressor(model=modelName, revision_method=revisionMethod, n_jobs=4)
    return clf.fit(dataFrame, hierarchy)

In [4]:
def prediction(model, steps=28):
    return model.predict(steps_ahead=steps)

In [5]:
def saveModel(path, model, method, ols, samples=None):
    
    dt = time.localtime()
    strinstante = "%02d-%02d-%04d-%02d-%02d-%02d" % (
    dt.tm_year, dt.tm_mon, dt.tm_mday, dt.tm_hour, dt.tm_min, dt.tm_sec)

    if (samples):
        filename = path + "model" + method + ols + str(samples) + strinstante + ".pkl"
    else:
        filename = path + "model" + method + ols + strinstante + ".pkl"
        
    with open(filename, 'wb') as file:
        pickle.dump(model, file, protocol=4)

In [6]:
def loadModel(path):
        
    with open(path, 'rb') as file:
        model = pickle.load(file)
        
    return model

In [7]:
def saveDataset(path, dataFrame, method, ols, samples=None):
    
    dt = time.localtime()
    strinstante = "%02d-%02d-%04d-%02d-%02d-%02d" % (
    dt.tm_year, dt.tm_mon, dt.tm_mday, dt.tm_hour, dt.tm_min, dt.tm_sec)

    if (samples):
        filename = path + "dataset" + method + ols + str(samples) + strinstante + ".pkl"
    else:
        filename = path + "dataset" + method + ols + strinstante + ".pkl"
        
    dataFrame.to_pickle(filename)

In [8]:
def plot(dictPlot, predictions, dataFrame):
    
    fig, axs = plt.subplots(len(dictPlot), figsize=(20, 30), sharex=True)
    # ax.grid(alpha=0.75)
    PLOT_FROM = 0
    dts = predictions[PLOT_FROM:].index

    for i, group in enumerate(dictPlot):
        axs[i].plot(dataFrame[PLOT_FROM:][group], 
                lw=1.1, 
                color='#2ecc71', 
                alpha=0.8, 
                label='Truth')
        axs[i].plot(predictions[PLOT_FROM:][group], 
                lw=1.1,
                color='#e74c3c', 
                alpha=0.8, 
                label='Prediction')
        axs[i].grid(alpha=0.75)
        axs[i].legend()
        axs[i].set_title(group)

In [9]:
def dataRangeSelection(df, start_date, end_date):
    #start_date = '2012-04-24'
    #end_date = '2016-04-24'
    mask = (df.index >= start_date) & (df.index <= end_date)    
    return df.loc[mask]

In [10]:
def rmse(y_true, y_pred):
    from sklearn.metrics import mean_squared_error

    results = {}
    for column in y_true.columns:
        results[column] = mean_squared_error(y_true[column], 
                                             y_pred[column], 
                                             squared=False)
    
    return pd.Series(results)

def mape(y_true, y_pred):
    from sklearn.metrics import mean_absolute_percentage_error

    results = {}
    for column in y_true.columns:
        results[column] = mean_absolute_percentage_error(y_true[column], 
                                                         y_pred[column])
    
    return pd.Series(results)

def mase(y_true, y_pred, y_train):
    from metrics import mean_absolute_scaled_error

    return mean_absolute_scaled_error(y_true, y_pred, y_train)

In [81]:
def erroBarGraf(dict_front_data):
    errorJson = {
            "legend":[
                      "RMSE",
                      "MAPE"
                   ],
             "name": [],
          
             "value":[
                      {
                         "RMSE":[],
                         "MAPE":[]
                      }
                   ]
        }
    for key in dict_front_data.keys():
        errorJson["name"].append(key)
        for k in dict_front_data[key].keys():
            errorJson["value"][0]['RMSE'].append(mean(list(dict_front_data[key][k]['RMSE'])))
            errorJson["value"][0]["MAPE"].append(mean(list(dict_front_data[key][k]['MAPE'])))
            
    
    return errorJson


def errorToJson(dict_data, revisionMethodName):
    errorRMSE = {"name": "RMSE",
                 "values":[]
                }
    
    errorMAPE = {"name": "MAPE",
                 "values":[]
                }
    errorRMSE["values"] = { key : list() for key in [revisionMethodName]} if isinstance(revisionMethodName, str) else { key : list() for key in revisionMethodName}
    errorMAPE["values"] = { key : list() for key in [revisionMethodName]} if isinstance(revisionMethodName, str) else { key : list() for key in revisionMethodName}
    
    for key in dict_data.keys():
        for k in dict_data[key].keys():
            errorRMSE["values"][k].append(mean(list(dict_data[key][k]['RMSE'])))
            errorMAPE["values"][k].append(mean(list(dict_data[key][k]['MAPE'])))
    
    return errorRMSE, errorMAPE


def jsonResult(self, modelsList, revisionMethodList, prediction, hist, error, _last=True):
    
        
    ht_tree = {
        'name':  self.key,
        'values': [],
    }

    ht_tree['values'] = ht_tree.fromkeys([modelsList, 'historical']) if isinstance(modelsList, str) else ht_tree.fromkeys(x) 
    ht_tree['values']['historical'] =  list(hist[self.key])
    if isinstance(modelsList, str):
        ht_tree['values'][modelsList] = ht_tree.fromkeys([revisionMethodList]) if isinstance(revisionMethodList, str) else ht_tree.fromkeys(revisionMethodList)
        for revisionKey in ht_tree['values'][modelsList].keys():
            ht_tree['values'][modelsList][revisionKey] = ht_tree.fromkeys(['prediction', 'error'])
            ht_tree['values'][modelsList][revisionKey]['prediction'] = list(prediction[modelsList][revisionKey][self.key])
            ht_tree['values'][modelsList][revisionKey]['error'] = ht_tree.fromkeys(error[modelsList][revisionKey].keys())
            for errorKey in ht_tree['values'][modelsList][revisionKey]['error'].keys():
                ht_tree['values'][modelsList][revisionKey]['error'][errorKey] = error[modelsList][revisionKey][errorKey][self.key]

    else:
        for modelKey in modelsList:
            ht_tree['values'][modelKey] = ht_tree.fromkeys([revisionMethodList]) if isinstance(revisionMethodList, str) else ht_tree.fromkeys(revisionMethodList)
            for revisionKey in ht_tree['values'][modelKey].keys():
                ht_tree['values'][modelKey][revisionKey] = ht_tree.fromkeys(['prediction', 'error'])
                ht_tree['values'][modelKey][revisionKey]['prediction'] = list(prediction[modelKey][revisionKey][self.key])
                ht_tree['values'][modelKey][revisionKey]['error'] = ht_tree.fromkeys(error[modelKey][revisionKey].keys())
                for errorKey in ht_tree['values'][modelKey][revisionKey]['error'].keys():
                    ht_tree['values'][modelKey][revisionKey]['error'][errorKey] = error[modelKey][revisionKey][errorKey][self.key]

    child_count = len(self.children)
    
    for i, child in enumerate(self.children):
        _last = i == (child_count - 1)
        if i == 0:
            ht_tree['nivel'] = []
            
        ht_tree['nivel'].append(jsonResult(child, modelsList, revisionMethodName, prediction, hist, error, _last=_last))
    return ht_tree


def jsonResultFront(self, modelsList, revisionMethodList, prediction, hist, _last=True):
    
        
    ht_tree = {
        "name":  self.key,
        "values": [],
    }
    ht_tree["values"].append(["historico", list(hist[self.key])])
    
    if isinstance(modelsList, str):
        for revisionKey in prediction[modelsList].keys():
            ht_tree["values"].append([modelsList + '-' + revisionKey,list(prediction[modelsList][revisionKey][self.key])]) 

    else:
        for modelKey in modelsList:
            for revisionKey in prediction[modelKey].keys():
                ht_tree["values"].append([modelKey + '-' + revisionKey,list(prediction[modelKey][revisionKey][self.key])])        
    
    child_count = len(self.children)
    
    for i, child in enumerate(self.children):
        _last = i == (child_count - 1)
        if i == 0:
            ht_tree["nivel"] = []
            
        ht_tree["nivel"].append(jsonResultFront(child, modelsList, revisionMethodName, prediction, hist, _last=_last))
    return ht_tree

## create Hierarchy Tree Front

In [12]:
def createHierarchyTree(self, _last=True):

    ht_tree = {
            'name':  self.key,
        }
    child_count = len(self.children)
    
    for i, child in enumerate(self.children):
        _last = i == (child_count - 1)
        if i == 0:
            ht_tree['children'] = []
            
        ht_tree['children'].append(createHierarchyTree(child, _last=_last))
    return ht_tree

## Run

In [13]:
dataset_path = 'data/'
data_set = M5(dataset_path, samples=50)

data_set.train_set

,CA,WI,TX,CA_4,WI_2,WI_3,TX_2,CA_1,TX_3,TX_1,...,TX_3_FOODS_2_055_TX_3_validation,CA_4_HOUSEHOLD_1_524_CA_4_validation,CA_1_HOBBIES_1_328_CA_1_validation,CA_1_HOBBIES_1_295_CA_1_validation,CA_4_HOBBIES_2_095_CA_4_validation,TX_3_HOBBIES_1_213_TX_3_validation,TX_1_HOBBIES_1_089_TX_1_validation,CA_2_HOBBIES_1_177_CA_2_validation,CA_3_FOODS_1_016_CA_3_validation,total
date,,,,,,,,,,,,,,,,,,,,,
2011-01-29,31,12,9,0,0,6,3,10,2,4,...,0,0,2,0,0,2,0,0,3,52
2011-01-30,72,3,25,0,0,1,14,40,2,9,...,0,0,0,36,0,0,0,0,0,100
2011-01-31,59,8,11,0,0,8,9,39,0,2,...,0,0,1,36,0,0,0,0,0,78
2011-02-01,8,7,16,0,1,4,6,3,0,10,...,0,0,1,0,0,0,0,0,0,31
2011-02-02,24,5,11,0,0,5,2,9,1,8,...,0,0,0,8,0,1,0,0,0,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-04-20,18,13,9,1,0,6,6,5,0,3,...,0,0,1,0,0,0,0,1,0,40
2016-04-21,22,19,18,5,0,9,9,11,2,7,...,0,0,0,7,0,0,1,0,0,59
2016-04-22,23,12,10,2,0,4,4,7,4,2,...,0,1,0,4,0,0,1,1,0,45


## Preprocess Data (Optional)

In [14]:
dfSelected = downcasting(data_set.train_set)

## Json Tree Front

In [15]:
ht = HierarchyTree.from_nodes(nodes=data_set.hierarchy, df=dfSelected)
ht_tree = createHierarchyTree(ht)

ht_tree

{'name': 'total',
 'children': [{'name': 'CA',
   'children': [{'name': 'CA_4',
     'children': [{'name': 'CA_4_FOODS',
       'children': [{'name': 'CA_4_FOODS_3',
         'children': [{'name': 'CA_4_FOODS_3_143_CA_4_validation'}]},
        {'name': 'CA_4_FOODS_2',
         'children': [{'name': 'CA_4_FOODS_2_185_CA_4_validation'}]}]},
      {'name': 'CA_4_HOBBIES',
       'children': [{'name': 'CA_4_HOBBIES_1',
         'children': [{'name': 'CA_4_HOBBIES_1_042_CA_4_validation'},
          {'name': 'CA_4_HOBBIES_1_241_CA_4_validation'}]},
        {'name': 'CA_4_HOBBIES_2',
         'children': [{'name': 'CA_4_HOBBIES_2_076_CA_4_validation'},
          {'name': 'CA_4_HOBBIES_2_095_CA_4_validation'}]}]},
      {'name': 'CA_4_HOUSEHOLD',
       'children': [{'name': 'CA_4_HOUSEHOLD_2',
         'children': [{'name': 'CA_4_HOUSEHOLD_2_412_CA_4_validation'}]},
        {'name': 'CA_4_HOUSEHOLD_1',
         'children': [{'name': 'CA_4_HOUSEHOLD_1_524_CA_4_validation'}]}]}]},
    {'name': 

# Saves 

In [296]:
path = 'result/'
method = '_sarimax'
ols = '_OLS_'
dataFrame = data_set.train_set
samples = data_set.sampleSize

saveDataset(path, dataFrame, method, ols, samples=samples)
saveModel(path, model, method, ols, samples=samples)

## Restore model

In [297]:
pathModel = 'result/model_sarimax_OLS_502021-12-0006-10-36-33.pkl'
modelRestore = loadModel(pathModel)

## Run Full process

In [61]:
with open('jsonFrontBack.json') as jsonfile:
    frontJson = json.load(jsonfile)
    
modelsListName = frontJson['modelo']
revisionMethodName = frontJson['tecnica']
previsionSteps = frontJson['previsao']
hierarchy = data_set.hierarchy
train, test = train_test_split(dfSelected, test_size=previsionSteps, shuffle=False)


errorListKeys = ['RMSE', 'MAPE']
dict_error = {}
dict_prediction = {}

dict_error =  dict_error.fromkeys([modelsListName]) if isinstance(modelsListName, str) else dict_error.fromkeys(modelsListName) 
dict_prediction =  dict_prediction.fromkeys([modelsListName]) if isinstance(modelsListName, str) else dict_prediction.fromkeys(modelsListName) 
    

In [62]:
if isinstance(modelsListName, str):
    dict_error[modelsListName] =  dict_error.fromkeys([revisionMethodName]) if isinstance(revisionMethodName, str) else dict_error.fromkeys(revisionMethodName) 
    dict_prediction[modelsListName] =  dict_prediction.fromkeys([revisionMethodName]) if isinstance(revisionMethodName, str) else dict_prediction.fromkeys(revisionMethodName)
    for key in dict_prediction[modelsListName].keys():
        modelTrained = runModel(modelsListName, hierarchy, train, revisionMethod=key)
        preds = prediction(modelTrained, previsionSteps)
        dict_prediction[modelsListName][key] = preds[-previsionSteps:]
        y_train = train
        y_true = test
        y_pred = preds[-previsionSteps:]
        
        dict_error[modelsListName][key] = dict_error.fromkeys([errorListKeys]) if isinstance(errorListKeys, str) else dict_error.fromkeys(errorListKeys) 
        dict_error[modelsListName][key]['RMSE'] = rmse(y_true, y_pred)
        dict_error[modelsListName][key]['MAPE'] = mape(y_true, y_pred)
        
else: 
    for model in modelsListName:
        dict_error[model] =  dict_error.fromkeys([revisionMethodName]) if isinstance(revisionMethodName, str) else dict_error.fromkeys(revisionMethodName) 
        dict_prediction[model] =  dict_prediction.fromkeys([revisionMethodName]) if isinstance(revisionMethodName, str) else dict_prediction.fromkeys(revisionMethodName) 
        for key in dict_prediction[model].keys():
            modelTrained = runModel(model, hierarchy, train, revisionMethod=key)
            preds = prediction(modelTrained, previsionSteps)
            dict_prediction[model][key] = preds[-previsionSteps:]
            y_train = train
            y_true = test
            y_pred = preds[-previsionSteps:]

            dict_error[model][key] = dict_error.fromkeys([errorListKeys]) if isinstance(errorListKeys, str) else dict_error.fromkeys(errorListKeys) 
            dict_error[model][key]['RMSE'] = rmse(y_true, y_pred)
            dict_error[model][key]['MAPE'] = mape(y_true, y_pred)

Fitting models:   0%|          | 0/18 [00:00<?, ?it/s]INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling dai

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
Fitting models:  50%|█████     | 9/18 [00:10<00:09,  1.10s/it]INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
Fitting models:  56%|█████▌    | 10/18 [00:10<00:07,  1.13it/s]INFO:fbprophet:Disabling daily seasonality. Run prophet wit

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
Fitting models:  11%|█         | 2/18 [00:04<00:26,  1.68s/it]INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
Fitting models:  22%|██▏       | 4/18 [00:04<00:09,  1.40it/s]INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
Fitting models:  72%|███████▏  | 13/18 [00:14<00:05,  1.13s/it]INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disa

In [87]:
with open('dictError.json','wb') as f:
    pickle.dump(dict_error, f, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('dictPrediction.json', 'wb') as f:
    pickle.dump(dict_prediction, f, protocol=pickle.HIGHEST_PROTOCOL)    

In [71]:
grafError = erroBarGraf(dict_error)

with open('grafBarError.json', 'w', encoding='utf-8') as f:
    json.dump(grafError, f, ensure_ascii=False, indent=4)

In [72]:
grafError

{'legend': ['RMSE', 'MAPE'],
 'name': ['auto_arima', 'prophet', 'holt_winters'],
 'value': [{'RMSE': [3.4241685848357233,
    3.436464397852943,
    4.086934555501098,
    4.181694094901999,
    3.9778774335357308,
    3.983870039073634],
   'MAPE': [4215723560625708.5,
    4290157893892732.5,
    4077498063162119.0,
    5105670784649972.0,
    3914186472684439.5,
    5241486569862485.0]}]}

In [73]:
errorRMSE, errorMAPE = errorToJson(dict_error, revisionMethodName)

In [74]:
with open('errorRMSE.json', 'w', encoding='utf-8') as f:
    json.dump(errorRMSE, f, ensure_ascii=False, indent=4)
    
with open('errorMAPE.json', 'w', encoding='utf-8') as f:
    json.dump(errorMAPE, f, ensure_ascii=False, indent=4)

In [75]:
errorRMSE

{'name': 'RMSE',
 'values': {'OLS': [3.4241685848357233, 4.086934555501098, 3.9778774335357308],
  'WLSS': [3.436464397852943, 4.181694094901999, 3.983870039073634]}}

In [88]:
errorMAPE

{'name': 'MAPE',
 'values': {'OLS': [4215723560625708.5,
   4077498063162119.0,
   3914186472684439.5],
  'WLSS': [4290157893892732.5, 5105670784649972.0, 5241486569862485.0]}}

In [79]:
finalJson = {
     "recorte": list( y_true.index.strftime("%Y-%m-%d")),
     "nivel": [jsonResultFront(ht, modelsListName, revisionMethodName, dict_prediction, y_true)]
 }

In [83]:
with open('result.json', 'w', encoding='utf-8') as f:
    json.dump(finalJson, f, ensure_ascii=False, indent=4)

In [80]:
finalJson

{'recorte': ['2016-03-28',
  '2016-03-29',
  '2016-03-30',
  '2016-03-31',
  '2016-04-01',
  '2016-04-02',
  '2016-04-03',
  '2016-04-04',
  '2016-04-05',
  '2016-04-06',
  '2016-04-07',
  '2016-04-08',
  '2016-04-09',
  '2016-04-10',
  '2016-04-11',
  '2016-04-12',
  '2016-04-13',
  '2016-04-14',
  '2016-04-15',
  '2016-04-16',
  '2016-04-17',
  '2016-04-18',
  '2016-04-19',
  '2016-04-20',
  '2016-04-21',
  '2016-04-22',
  '2016-04-23',
  '2016-04-24'],
 'nivel': [{'name': 'total',
   'values': [['historico',
     [47,
      29,
      33,
      48,
      69,
      66,
      73,
      50,
      70,
      46,
      51,
      47,
      73,
      75,
      43,
      41,
      40,
      34,
      47,
      100,
      87,
      48,
      53,
      40,
      59,
      45,
      50,
      56]],
    ['auto_arima-OLS',
     [30.384343147706907,
      32.49621889702048,
      32.91476899445957,
      33.10464168083871,
      33.1504426865644,
      33.16557868092129,
      33.111804431173276,
 